<a href="https://colab.research.google.com/github/mikecchau/Utilizing-LinearSVM-for-stroke-prediction/blob/main/SVM_stroke_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git init

Reinitialized existing Git repository in /content/.git/


In [5]:
!git clone https://github.com/mikecchau/Utilizing-LinearSVM-for-stroke-prediction

Cloning into 'Utilizing-LinearSVM-for-stroke-prediction'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [11]:
%cd /content/Utilizing-LinearSVM-for-stroke-prediction/Data

/content/Utilizing-LinearSVM-for-stroke-prediction/Data


In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
df=pd.read_csv('healthcare-dataset-stroke-data.csv')

In [21]:
df.shape

(5110, 12)

In [23]:
df = df.drop('id', axis=1)


Drop 'id' column

In [37]:
df.tail(20)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
5070,Female,5.00,0,0,No,children,Urban,61.98,16.800000,Unknown,0
5071,Male,81.00,0,0,Yes,Private,Rural,135.32,35.800000,Unknown,0
5072,Female,35.00,0,0,Yes,Private,Rural,58.72,40.000000,smokes,0
5073,Female,24.00,0,0,No,Private,Urban,90.42,24.300000,never smoked,0
5074,Female,52.00,0,0,Yes,Govt_job,Urban,104.00,25.600000,smokes,0
5075,Male,70.00,0,0,Yes,Private,Urban,102.50,37.800000,Unknown,0
5076,Female,34.00,0,0,Yes,Private,Rural,174.37,23.000000,never smoked,0
5077,Male,17.00,0,0,No,Private,Rural,106.56,21.000000,Unknown,0
5078,Female,8.00,0,0,No,children,Urban,76.31,15.500000,Unknown,0
5079,Male,1.72,0,0,No,children,Urban,77.28,17.100000,Unknown,0


In [17]:
from sklearn.impute import SimpleImputer

In [25]:
df['stroke'].isnull().sum()

0

There are no missing labels in the dataset

In [27]:
# Create features matrix
X = df.drop('stroke', axis=1)

In [28]:
# Create label matrix
y = df['stroke']

In [31]:
# Check for missing values in columns
for x in X:
  print(df[x].isnull().sum())

0
0
0
0
0
0
0
0
201
0


This indicates that there are 201 missing values within the 'bmi' column.

In [33]:
# Fill missing 'bmi' values with mean
df['bmi'].fillna(value=df['bmi'].mean(), inplace=True)

In [35]:
# Recheck for missing values
df.isnull().values.any()

False